In [1]:
import pandas
from datetime import datetime, timedelta
import time

In [2]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, roc_auc_score

In [4]:
chats = pandas.read_csv("train\\chats_train.csv").sort_values('timestamp')

In [5]:
news = pandas.read_csv("train\\news_train.csv")
onchain_EOS = pandas.read_csv("train\\onchain\\EOS_train.csv")
onchain_TRX = pandas.read_csv("train\\onchain\\TRX_train.csv")
prices = pandas.read_csv("train\\tickers_train.csv")

In [6]:
ticker = 'ZRX'
delta = timedelta(hours=2)

In [7]:
#Найдем максимальную и минимальную даты в файле с ценами
min_date = datetime.today()
max_date = datetime.fromordinal(1)
# отфильтруем price по нашему токену и отсортируем по времени
tic_prices = prices.loc[prices['ticker']==ticker].sort_values('datetime',ascending=True)

In [8]:
for line in tic_prices['datetime']:
    curtime = datetime.strptime(line,"%Y-%m-%dT%H:%M:%SZ")
    if curtime > max_date:
        max_date = curtime
    if curtime < min_date:
        min_date = curtime

In [9]:
print(min_date, max_date)

2018-03-01 00:00:00 2018-04-30 22:00:00


In [12]:
Y = []
price_index = 0
cur_date = min_date
last_price = tic_prices['priceBtc'].iloc[0]
while cur_date <= max_date:
    cur_price = tic_prices.iloc[price_index]
    price_time = datetime.strptime(cur_price['datetime'],"%Y-%m-%dT%H:%M:%SZ")
    #print(price_index, price_time,cur_date, last_price)
    if price_time == cur_date:
        Y.append(1 if cur_price['priceBtc'] < last_price else 0)
        last_price = cur_price['priceBtc']
        price_index = price_index + 1
    else:
        Y.append(0) 
    cur_date = cur_date + delta


In [13]:
chats_index = 0
#Пропустим все тексты, которые появились раньше, чем за 2 часа до первой котировки
first_text_timestamp = datetime.timestamp(min_date - delta) 
while chats.iloc[chats_index]['timestamp'] < first_text_timestamp:
    chats_index = chats_index + 1
print(chats_index)
cur_day = min_date.day
cur_chats = []
cur_news = []
cur_date = min_date
X_raw = []
X = []
while cur_date <= max_date:
    while (chats.iloc[chats_index]['timestamp'] < datetime.timestamp(cur_date)):
        if chats.iloc[chats_index]['sender_is_bot']:            
            chats_index = chats_index + 1
            continue
        cur_chats.append(chats.iloc[chats_index][['chat_id', 'chat_title', 'emoji_list', 'sender_first_name', 'sender_id',
       'sender_is_bot', 'sender_last_name', 'sender_username', 'text']])
        chats_index = chats_index + 1
        cur_chats_counter = cur_chats_counter + 1
        if cur_chats_counter > 200:
            break
    X_raw.append(cur_chats.copy())
    cur_chats = []
    cur_chats_counter = 0
    cur_date = cur_date + delta
    #if cur_date.day != cur_day:
        #print(cur_date)
        #cur_day = cur_date.day

0


In [14]:
X = []
for cur_X in X_raw:
    cur_str = ''
    for message in cur_X:
        cur_str = cur_str + str(message['text']) + ' '
    X.append(cur_str)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=41, stratify=Y)
print(len(x_train), len(x_test), len(y_train), len(y_test))

658 74 658 74


In [17]:
vectorizer_char = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='word', 
                             ngram_range=(1, 2))

vectorizer_char.fit(x_train);

tfidf_matrix_char_train = vectorizer_char.transform(x_train)
tfidf_matrix_char_test = vectorizer_char.transform(x_test)

lr_char = LogisticRegression(solver='sag', verbose=2, multi_class='multinomial')
#lr_char = RandomForestClassifier()
lr_char.fit(tfidf_matrix_char_train, y_train)

y_pred_char = lr_char.predict(tfidf_matrix_char_test)
print(accuracy_score(y_test, y_pred_char))
y_pred_train = lr_char.predict(tfidf_matrix_char_train)
print(accuracy_score(y_train, y_pred_train))


convergence after 24 epochs took 1 seconds
0.5
0.9969604863221885


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [18]:
acc = [0,0]
for i in range (len(y_test)):
    if y_test[i] == y_pred_char[i]:
        acc[y_pred_char[i]] = acc[y_pred_char[i]] + 1
print(acc, len(y_test))

[24, 13] 74
